In [ ]:
import os
import requests
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Get the API key from the environment variable
API_KEY = os.getenv("CONGRESS_API_KEY")
print(API_KEY)

In [ ]:
# Define the API endpoint and parameters
BASE_URL = "https://api.congress.gov/v3"
endpoint = "/bill"
congresses = [i for i in range(106, 120)] # year 2000-2025
congresses.reverse()
bill_types = ["hr", "s", "hjres", "sjres"] # house of reps, senate, house joint res, senate joint res
params = {"api_key" : API_KEY,
          "limit" : 1} # 250 is max
congresses

In [ ]:
import json
import time

def check_for_rate_limit(r_json):
    if "error" in r_json and "code" in r_json["error"]:
        if r_json["error"]["code"] == "OVER_RATE_LIMIT":
            print("OVER_RATE_LIMIT: sleeping for 1800")
            time.sleep(1800) # sleep 30 min
            return True
    return False

done = [119]

# get counts for each bill type in a given congress
for c in congresses:
    if c in done: # skip
        continue
    congress = f"/{c}"
    
    for b_t in bill_types:
        params["limit"] = 1 # min
        bills = {}
        bill_type = f"/{b_t}"
        url = f"{BASE_URL}{endpoint}{congress}{bill_type}"
        response = requests.get(url, params=params)
        r_json = json.loads(response.text)
        while check_for_rate_limit(r_json):
            response = requests.get(url, params=params)
            r_json = json.loads(response.text)
        print(r_json)
        total_bills_in_bill_type = r_json["pagination"]["count"]
        
        for i in range(1, total_bills_in_bill_type + 1):
            try:
                params["limit"] = 250 # max
                print(c, b_t, i, total_bills_in_bill_type)
                key = f"{c}_{b_t}_{i}"
                tmp_dict = {}

                bill_url = f"{BASE_URL}{endpoint}{congress}{bill_type}/{i}"
                response = requests.get(bill_url, params=params)
                r_json = json.loads(response.text)
                while check_for_rate_limit(r_json):
                    response = requests.get(url, params=params)
                    r_json = json.loads(response.text)
                tmp_dict["bill"] = r_json

                actions_url = f"{BASE_URL}{endpoint}{congress}{bill_type}/{i}/actions"
                response = requests.get(actions_url, params=params)
                r_json = json.loads(response.text)
                while check_for_rate_limit(r_json):
                    response = requests.get(url, params=params)
                    r_json = json.loads(response.text)
                tmp_dict["actions"] = r_json
                try:
                    if "pagination" in tmp_dict["actions"]: # if actions have been taken
                        if tmp_dict["actions"]["pagination"]["count"] > 250: # account for > 250 actions for a bill
                            j = 251
                            while j < tmp_dict["actions"]["pagination"]["count"]:
                                params["offset"] = j
                                response = requests.get(actions_url, params=params)
                                tmp_actions = json.loads(response.text)
                                while check_for_rate_limit(tmp_actions):
                                    response = requests.get(url, params=params)
                                    tmp_actions = json.loads(response.text)
                                tmp_actions = tmp_actions["actions"]
                                for action in tmp_actions:
                                    tmp_dict["actions"]["actions"][action] = tmp_actions[action]
                                j += 250
                            del params["offset"] # remove key
                except Exception as e:
                    error_message = f"ERROR: {e}\nTYPE: {type(e)}"
                    with open(f"{c}_{b_t}_{i}.txt", "w") as f:
                        f.write(error_message)

                bill_text_url = f"{BASE_URL}{endpoint}{congress}{bill_type}/{i}/text" # change this to get bill text
                response = requests.get(bill_text_url, params=params)
                r_json = json.loads(response.text)
                while check_for_rate_limit(r_json):
                    response = requests.get(url, params=params)
                    r_json = json.loads(response.text)
                tmp_dict["text"] = r_json
                # if "textVersions" in tmp_dict["text"]:
                #     for v in tmp_dict["text"]["textVersions"]:
                #         response = requests.get(v["formats"][0]["url"], params=params)
                #         # r_json = json.loads(response.text)
                #         # while check_for_rate_limit(r_json):
                #         #     response = requests.get(url, params=params)
                #         #     r_json = json.loads(response.text)
                #         tmp_dict["text"][v["date"]] = response.text
            except Exception as e:
                error_message = f"ERROR: {e}\nTYPE: {type(e)}"
                tmp_dict = {}
                tmp_dict["error"] = error_message
            bills[key] = tmp_dict
            
        with open(f"{c}_{b_t}.json", "w") as f:
            json.dump(bills, f)